# Instacart Feature Engineering

In this notebook, we will engineer the target variable and five explanatory variables.

#### The target variable

We will create a list of unique pairs of users and products from the `prior` set of orders.  We will use this list, which we label as "`up_pair`", as the index of our final data table.

For each unique pair of user and product from the `prior` set of orders, if the user bought the product in the  `prior` set of orders *and* in the `train` set of orders, then our target variable `y` is assigned the value 1.  On the other hand, if the user bought the product in the  `prior` set, but they didn't buy the product in the `train` set, then our target variable `y` is assigned the value 0.


#### The explanatory variables

The five explanatory features that we engineer are as follows:

Given the user-product pair of (User A, Product B),

1. `total_buy_n5`: the total number of times User A bought Product B out of the 5 most recent orders.

2. `total_buy_ratio_n5`: the proportion of User A's 5 most recent orders in which User A bought Product B.

3. `order_ratio_by_chance_n5`: the proportion of User A's 5 most recent orders in which User A had the "chance" to buy B, and did indeed do so.  Here, a "chance" refers to the number of opportunities the user had for buying the item after first encountering (*viz.*, buying)  it. For example, if a User A bought Product B for the first time in their 4th most recent order, then the user would have had 4 chances to buy the product.  If that user had bought the product only in their 4th and 2nd most recent orders, then `order_ratio_by_chance_n5` would be 0.5 (*i.e., (1+1)/4) for that user-product pair.

4. `useritem_order_days_max_n5`: the longest number of days that User A has recently gone without buying Product B.  We are only considering the 5 most recent orders.

5. `useritem_order_days_min_n5`: the shortest number of days that User A has recently gone without buying Product B.  Again, we are only considering the 5 most recent orders.


The choice of these five features was inspired by [Onodera's solution](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/]), which won 2nd place in the Instacart Kaggle competition.

#### Comments on notation:

In what follows, 
+ "`_n5`" *generally* stands for the 5 most recent orders.
+ "`_n1`" stands for the most recent order.
+ "`_n2`" stands for the second most recent order.
+ ...
+ "`_n5`" *sometimes* stands for the fifth most recent order.


### Import libraries and modify notebook settings

In [2]:
# Import libraries
import os
import sys
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.engine.reflection import Inspector
import csv
import pandas as pd
from shutil import copy2
#import datetime

# Modify notebook settings
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

### Create paths to data folders and files

In [3]:
# Create a variable for the project root directory
proj_root = os.path.join(os.pardir)

# Save the path to the folder that will contain the intermediate data 
# that will be transformed: /data/interim
interim_data_dir = os.path.join(proj_root,
                                "data",
                                "interim")

# Save the path to the SQLite databse containing the untransformed
# Instacart data
db_name = 'instacart_2017_05_01.db'

interim_sqlitedb_path = os.path.join(interim_data_dir,
                                     db_name)

interim_sqlitedb_eng = os.path.join('sqlite:///',
                                    interim_sqlitedb_path)

# Save the path to the SQLite database that will contain the transformed
# Instacart data
db_name_tf = 'instacart_transformed.db'

transformed_sqlitedb_path = os.path.join(interim_data_dir,
                                         db_name_tf)

# Save the engine path to the SQLite database that will contain the 
# transformed Instacart data
transformed_sqlitedb_eng = os.path.join('sqlite:///',
                                        transformed_sqlitedb_path)


# Save the path to the folder that will contain the final,
# processed data: /data/processed
processed_data_dir = os.path.join(proj_root,
                                "data",
                                "processed")

# Save the path to the csv file that will contain the final,
# processed Instacart data
final_csv_name = 'instacart_final.csv'

final_csv_path = os.path.join(processed_data_dir,
                              final_csv_name)

### Make a copy of the untransformed database.
We will transform this copy while leaving the first database untampered.

In [3]:
copy2(interim_sqlitedb_path, transformed_sqlitedb_path)

'../data/interim/instacart_transformed.db'

### Read data from sqlite database

In [4]:
# Create a sqlite3 connection cursor
conn = sqlite3.connect(transformed_sqlitedb_path)
c = conn.cursor()

In [5]:
# Create an engine to the new SQLite database 
engine = create_engine(transformed_sqlitedb_eng, echo=False)

In [6]:
# List table names
engine.table_names()

['aisles',
 'departments',
 'order_products__prior',
 'order_products__train',
 'orders',
 'products']

In [7]:
# List view names
Inspector.from_engine(engine).get_view_names()

[]

### Transform the data and create new views and tables

In [8]:
# Create table
c.execute('''CREATE TABLE up_pairs_train AS
             SELECT substr('00'||orders.user_id, -6) || '-' || 
                 substr('0000'||order_products__train.product_id, -6) AS up_pair
             FROM order_products__train
             JOIN orders
             ON orders.order_id = order_products__train.order_id
             GROUP BY up_pair
             ORDER BY user_id ASC, product_id ASC''')

# Save (commit) the changes
conn.commit()

In [9]:
# Create table
c.execute('''CREATE TABLE up_pairs_prior AS
             SELECT substr('00'||orders.user_id, -6) || '-' || 
                 substr('0000'||order_products__prior.product_id, -6) AS up_pair,
                 orders.user_id AS user_id, 
                 order_products__prior.product_id AS product_id,
                 orders.order_number AS order_number
             FROM order_products__prior
             JOIN orders
             ON orders.order_id = order_products__prior.order_id
             ORDER BY user_id ASC, product_id ASC''')

# Save (commit) the changes
conn.commit()

In [10]:
# Create table
c.execute('''CREATE TABLE max_order_by_user AS
             SELECT user_id, MAX(order_number) AS max_order_number
             FROM up_pairs_prior
             GROUP BY user_id
             ORDER BY user_id ASC''')

# Save (commit) the changes
conn.commit()

In [11]:
# Create table
c.execute('''CREATE TABLE up_pairs_prior_modified AS
             SELECT up_pairs_prior.*, 
             max_order_by_user.max_order_number,
             (1 + max_order_by_user.max_order_number - up_pairs_prior.order_number) AS order_number_rev
             FROM up_pairs_prior
             JOIN max_order_by_user
             ON max_order_by_user.user_id = up_pairs_prior.user_id''')

# Save (commit) the changes
conn.commit()

In [12]:
# Create table
c.execute('''CREATE TABLE first_data_table AS
             SELECT up_pairs_prior_modified.up_pair AS up_pair,
                 up_pairs_prior_modified.user_id AS user_id, 
                 up_pairs_prior_modified.product_id AS product_id, 
                 CASE WHEN up_pairs_train.up_pair IS NULL THEN 0 ELSE 1 END as 'y',
                 SUM(CASE WHEN (order_number_rev <= 5) THEN 1 ELSE 0 END) AS total_buy_n5,    
                 SUM(CASE WHEN (order_number_rev <= 5) THEN 1 ELSE 0 END) / 5.0 AS total_buy_ratio_n5,                 
                 MAX(order_number_rev) AS max_order_number_rev,
                 CASE WHEN MAX(order_number_rev) > 5
                     THEN (SUM(CASE WHEN (order_number_rev <= 5) THEN 1 ELSE 0 END) / 5.0)
                     ELSE (SUM(CASE WHEN (order_number_rev <= 5) THEN 1 ELSE 0 END) /
                         (MAX(order_number_rev) * 1.0))
                     END AS order_ratio_by_chance_n5                     
             FROM up_pairs_prior_modified
             LEFT JOIN up_pairs_train ON up_pairs_train.up_pair = up_pairs_prior_modified.up_pair
             GROUP BY up_pairs_prior_modified.up_pair''')

# Save (commit) the changes
conn.commit()

In [13]:
# Create table
c.execute('''CREATE VIEW n5_view AS
             SELECT user_id, 
                 MAX(order_number) AS order_number_n1,
                 CASE WHEN (MAX(order_number) - 1) < 1 THEN NULL 
                     ELSE (MAX(order_number) - 1) END AS order_number_n2,
                 CASE WHEN (MAX(order_number) - 2) < 1 THEN NULL 
                     ELSE (MAX(order_number) - 2) END AS order_number_n3,
                 CASE WHEN (MAX(order_number) - 3) < 1 THEN NULL 
                     ELSE (MAX(order_number) - 3) END AS order_number_n4,
                 CASE WHEN (MAX(order_number) - 4) < 1 THEN NULL 
                     ELSE (MAX(order_number) - 4) END AS order_number_n5
             FROM orders
             WHERE eval_set = 'prior'
             GROUP BY user_id''')

# Save (commit) the changes
conn.commit()

In [14]:
# Create table
c.execute('''CREATE TABLE n5_table AS
             SELECT n5_view.user_id,
                 orders_t1.order_id AS order_id_n1,
                 orders_t2.order_id AS order_id_n2,
                 orders_t3.order_id AS order_id_n3,
                 orders_t4.order_id AS order_id_n4,
                 orders_t5.order_id AS order_id_n5,
                 orders_t1.days_since_prior_order AS days_since_prior_order_n1,
                 orders_t2.days_since_prior_order AS days_since_prior_order_n2,
                 orders_t3.days_since_prior_order AS days_since_prior_order_n3,
                 orders_t4.days_since_prior_order AS days_since_prior_order_n4,
                 orders_t5.days_since_prior_order AS days_since_prior_order_n5
             FROM n5_view
                 LEFT JOIN orders AS orders_t1
                 ON (orders_t1.user_id = n5_view.user_id
                     AND orders_t1.order_number = n5_view.order_number_n1)
                 LEFT JOIN orders AS orders_t2
                 ON (orders_t2.user_id = n5_view.user_id
                     AND orders_t2.order_number = n5_view.order_number_n2)
                 LEFT JOIN orders AS orders_t3
                 ON (orders_t3.user_id = n5_view.user_id
                     AND orders_t3.order_number = n5_view.order_number_n3)
                 LEFT JOIN orders AS orders_t4
                 ON (orders_t4.user_id = n5_view.user_id
                     AND orders_t4.order_number = n5_view.order_number_n4)
                 LEFT JOIN orders AS orders_t5
                 ON (orders_t5.user_id = n5_view.user_id
                     AND orders_t5.order_number = n5_view.order_number_n5)''')

# Save (commit) the changes
conn.commit()

In [15]:
# Create table
c.execute('''CREATE TABLE new_table AS
             SELECT first_data_table.*,
                 n5_table.order_id_n1,
                 n5_table.order_id_n2,
                 n5_table.order_id_n3,
                 n5_table.order_id_n4,
                 n5_table.order_id_n5
             FROM first_data_table
             LEFT JOIN n5_table
             ON (n5_table.user_id = first_data_table.user_id)
             ''')

# Save (commit) the changes
conn.commit()

In [16]:
# Create table
c.execute('''CREATE TABLE new_table_2 AS
             SELECT new_table.up_pair,
                 new_table.y,
                 new_table.total_buy_n5,
                 new_table.total_buy_ratio_n5,
                 new_table.order_ratio_by_chance_n5,                 
                 n5_table.days_since_prior_order_n1,
                 n5_table.days_since_prior_order_n2,
                 n5_table.days_since_prior_order_n3,
                 n5_table.days_since_prior_order_n4,
                 n5_table.days_since_prior_order_n5,                 
                 CASE WHEN order_products__prior_1.product_id IS NULL THEN 0 ELSE 1 END as bought_n1,
                 CASE WHEN order_products__prior_2.product_id IS NULL THEN 0 ELSE 1 END as bought_n2,
                 CASE WHEN order_products__prior_3.product_id IS NULL THEN 0 ELSE 1 END as bought_n3,
                 CASE WHEN order_products__prior_4.product_id IS NULL THEN 0 ELSE 1 END as bought_n4,
                 CASE WHEN order_products__prior_5.product_id IS NULL THEN 0 ELSE 1 END as bought_n5
             FROM new_table
             LEFT JOIN n5_table
                 ON (n5_table.user_id = new_table.user_id)
             LEFT JOIN order_products__prior AS order_products__prior_1
                 ON (order_products__prior_1.order_id = new_table.order_id_n1
                     AND order_products__prior_1.product_id = new_table.product_id)
             LEFT JOIN order_products__prior AS order_products__prior_2
                 ON (order_products__prior_2.order_id = new_table.order_id_n2
                     AND order_products__prior_2.product_id = new_table.product_id)
             LEFT JOIN order_products__prior AS order_products__prior_3
                 ON (order_products__prior_3.order_id = new_table.order_id_n3
                     AND order_products__prior_3.product_id = new_table.product_id)
             LEFT JOIN order_products__prior AS order_products__prior_4
                 ON (order_products__prior_4.order_id = new_table.order_id_n4
                     AND order_products__prior_4.product_id = new_table.product_id)
             LEFT JOIN order_products__prior AS order_products__prior_5
                 ON (order_products__prior_5.order_id = new_table.order_id_n5
                     AND order_products__prior_5.product_id = new_table.product_id)
             ''')

# Save (commit) the changes
conn.commit()

In [17]:
# Create table
c.execute('''CREATE TABLE new_table_3 AS
             SELECT up_pair, 
                 y, 
                 total_buy_n5, 
                 total_buy_ratio_n5, 
                 order_ratio_by_chance_n5,                 
                 
                 MAX(IFNULL(order_days_n1, 0), 
                     IFNULL(order_days_n2, 0),
                     IFNULL(order_days_n3, 0),
                     IFNULL(order_days_n4, 0),
                     IFNULL(order_days_n5, 0)) AS useritem_order_days_max_n5,
                     
                 CASE WHEN days_since_prior_order_n5 IS NULL THEN
                     MIN(IFNULL(order_days_n1, 1000000), 
                         IFNULL(order_days_n2, 1000000),
                         IFNULL(order_days_n3, 1000000),
                         IFNULL(order_days_n4, 1000000),
                         MAX(IFNULL(order_days_n1, 0), 
                             IFNULL(order_days_n2, 0),
                             IFNULL(order_days_n3, 0),
                             IFNULL(order_days_n4, 0),
                             IFNULL(order_days_n5, 0)))  
                     ELSE
                     MIN(IFNULL(order_days_n1, 1000000), 
                         IFNULL(order_days_n2, 1000000),
                         IFNULL(order_days_n3, 1000000),
                         IFNULL(order_days_n4, 1000000),
                         order_days_n5)
                     END AS useritem_order_days_min_n5

             FROM 
             (
                 SELECT *,
                     CASE WHEN bought_n1 = 0 THEN NULL
                         ELSE days_since_prior_order_n1
                         END AS order_days_n1,
                 
                     CASE WHEN bought_n2 = 0 THEN NULL
                         ELSE CASE WHEN bought_n1 = 0 THEN
                             (days_since_prior_order_n1 + days_since_prior_order_n2)
                             ELSE days_since_prior_order_n2
                             END
                         END AS order_days_n2,
                 
                     CASE WHEN bought_n3 = 0 THEN NULL
                         ELSE 
                         CASE WHEN bought_n2 = 0 THEN 
                             CASE WHEN bought_n1 = 0 THEN
                                 (days_since_prior_order_n1 + days_since_prior_order_n2 +
                                 days_since_prior_order_n3)
                                 ELSE (days_since_prior_order_n2 + days_since_prior_order_n3) 
                                 END                    
                             ELSE days_since_prior_order_n3
                             END
                         END AS order_days_n3,
                         
                     CASE WHEN bought_n4 = 0 THEN NULL
                         ELSE 
                         CASE WHEN bought_n3 = 0 THEN
                             CASE WHEN bought_n2 = 0 THEN
                                 CASE WHEN bought_n1 = 0 THEN
                                     (days_since_prior_order_n1 + days_since_prior_order_n2 +
                                         days_since_prior_order_n3 + days_since_prior_order_n4)
                                     ELSE (days_since_prior_order_n2 + days_since_prior_order_n3 +
                                         days_since_prior_order_n4) 
                                     END                    
                                 ELSE (days_since_prior_order_n3 + days_since_prior_order_n4)
                                 END
                             ELSE days_since_prior_order_n4
                             END
                         END AS order_days_n4,
                         
                     CASE WHEN bought_n4 = 0 THEN
                         CASE WHEN bought_n3 = 0 THEN
                             CASE WHEN bought_n2 = 0 THEN
                                 CASE WHEN bought_n1 = 0 THEN
                                     (days_since_prior_order_n1 + days_since_prior_order_n2 +
                                         days_since_prior_order_n3 + days_since_prior_order_n4 + 
                                         IFNULL(days_since_prior_order_n5, 0))
                                     ELSE (days_since_prior_order_n2 + days_since_prior_order_n3 +
                                         days_since_prior_order_n4 + 
                                         IFNULL(days_since_prior_order_n5, 0)) 
                                     END                    
                                 ELSE (days_since_prior_order_n3 + days_since_prior_order_n4 + 
                                     IFNULL(days_since_prior_order_n5, 0))
                                 END
                             ELSE (days_since_prior_order_n4 + IFNULL(days_since_prior_order_n5, 0))
                             END
                         ELSE IFNULL(days_since_prior_order_n5, 0)
                         END AS order_days_n5
             
                 FROM new_table_2
             )
             GROUP BY up_pair
             ''')

# Save (commit) the changes
conn.commit()

# Save `new_table_3` to csv file as `final_data_table`

In [18]:
sql_query = \
"""
SELECT *
FROM new_table_3
LIMIT 5
"""

temp_df = pd.read_sql_query(sql=sql_query, con=engine)
temp_df

,up_pair,y,total_buy_n5,total_buy_ratio_n5,order_ratio_by_chance_n5,useritem_order_days_max_n5,useritem_order_days_min_n5
0,001-000196,1,5,1.0,1.0,30.0,0.0
1,001-010258,1,5,1.0,1.0,30.0,0.0
2,001-010326,0,0,0.0,0.0,83.0,83.0
3,001-012427,0,5,1.0,1.0,30.0,0.0
4,001-013032,1,2,0.4,0.4,34.0,19.0


In [19]:
# Create a list of column names
col_names = [x['name'] for x in Inspector.from_engine(engine).get_columns('new_table_3')]
col_names

['up_pair',
 'y',
 'total_buy_n5',
 'total_buy_ratio_n5',
 'order_ratio_by_chance_n5',
 'useritem_order_days_max_n5',
 'useritem_order_days_min_n5']

In [20]:
# Create the csv file that will contain the final, processed Instacart data
data = c.execute("SELECT * FROM new_table_3")
with open(final_csv_path, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(col_names)
    writer.writerows(data)

### Check that it wrote to the file.

In [6]:
temp_df = pd.read_csv(final_csv_path, 
                      index_col=0, 
                      nrows=10).head(10)
temp_df.head(10)

,y,total_buy_n5,total_buy_ratio_n5,order_ratio_by_chance_n5,useritem_order_days_max_n5,useritem_order_days_min_n5
up_pair,,,,,,
001-000196,1,5,1.0,1.0,30.0,0.0
001-010258,1,5,1.0,1.0,30.0,0.0
001-010326,0,0,0.0,0.0,83.0,83.0
001-012427,0,5,1.0,1.0,30.0,0.0
001-013032,1,2,0.4,0.4,34.0,19.0
001-013176,0,0,0.0,0.0,83.0,83.0
001-014084,0,0,0.0,0.0,83.0,83.0
001-017122,0,0,0.0,0.0,83.0,83.0
001-025133,1,5,1.0,1.0,30.0,0.0
